<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Intro-to-Keras" data-toc-modified-id="Intro-to-Keras-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Intro to <code>Keras</code></a></span></li><li><span><a href="#Installing-Keras" data-toc-modified-id="Installing-Keras-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Installing <code>Keras</code></a></span><ul class="toc-item"><li><span><a href="#Using-the-GPU" data-toc-modified-id="Using-the-GPU-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Using the GPU</a></span></li></ul></li><li><span><a href="#Building-Keras-Models-:-ANN-Example" data-toc-modified-id="Building-Keras-Models-:-ANN-Example-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Building <code>Keras</code> Models : ANN Example</a></span><ul class="toc-item"><li><span><a href="#The-Sequential-API" data-toc-modified-id="The-Sequential-API-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>The Sequential API</a></span></li><li><span><a href="#The-Functional-API" data-toc-modified-id="The-Functional-API-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>The Functional API</a></span></li></ul></li><li><span><a href="#Training-Keras-Models" data-toc-modified-id="Training-Keras-Models-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Training <code>Keras</code> Models</a></span><ul class="toc-item"><li><span><a href="#Compiling-the-Network" data-toc-modified-id="Compiling-the-Network-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Compiling the Network</a></span></li><li><span><a href="#Training-the-Network" data-toc-modified-id="Training-the-Network-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Training the Network</a></span></li><li><span><a href="#Evaluating-and-Making-Predictions" data-toc-modified-id="Evaluating-and-Making-Predictions-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Evaluating and Making Predictions</a></span></li></ul></li><li><span><a href="#Advanced-Keras-Layers" data-toc-modified-id="Advanced-Keras-Layers-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Advanced <code>Keras</code> Layers</a></span><ul class="toc-item"><li><span><a href="#Dropout-Layers" data-toc-modified-id="Dropout-Layers-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Dropout Layers</a></span></li><li><span><a href="#Merge-Layers" data-toc-modified-id="Merge-Layers-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Merge Layers</a></span></li><li><span><a href="#And-More..." data-toc-modified-id="And-More...-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>And More...</a></span></li></ul></li></ul></div>

# Imports

In [1]:
# some standard imports
import pandas as pd 
import numpy as np
import random

# keras specific
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Dropout, Input
from keras.layers import Concatenate
from keras.utils import np_utils
from keras.utils.np_utils import to_categorical
from keras.utils.data_utils import get_file
from keras.preprocessing.text import Tokenizer
from keras.utils.vis_utils import model_to_dot, plot_model
from keras.datasets import imdb, reuters
from keras.preprocessing import sequence
from keras.optimizers import SGD, RMSprop

np.random.seed(22)


Using TensorFlow backend.
/Users/samiahmed/anaconda3/envs/metis/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/samiahmed/anaconda3/envs/metis/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/samiahmed/anaconda3/envs/metis/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/sami

# Intro to `Keras` 

Keras is a high level neural networks API written in Python - capable of running on any major OS (Windows, MAC, and Linux compatible). It is a wrapper to Tensorflow.

# Installing `Keras`

Run this at the command line:

```$ conda install -c conda-forge keras```

If that doesn't work:

```$ conda install tensorflow```

... move into a folder for installing tools

```$ git clone https://github.com/fchollet/keras.git```

```$ cd keras```

```$ python setup.py install```

You can run this on your laptop, but it will be very slow. If you have a CUDA 7.5 compatable GPU (or use an AWS instance with one) keras will run way faster.

## Using the GPU

Deep Learning requires tons of matrix computations. GPUs can do these really efficiently. 

If you're working on a computer with a GPU and want to configure it for keras:
  - Install [CUDA](http://docs.nvidia.com/cuda/index.html#axzz4Pa5zY8Qi) 
  - In `.bashrc`/`.bash_profile`: `export THEANO_FLAGS=device=gpu,floatX=float32`

But if not, don't worry, you can leverage virtual machine images, like an AMI from AWS. **When using AWS, a Deep Learning AMI such as [this one](https://aws.amazon.com/marketplace/pp/Amazon-Web-Services-AWS-Deep-Learning-AMI-Ubuntu-1/B07Y43P7X5), is highly recommended**-- use an AMI to get all the packages you need (including CUDA setup) handled up front.

# Building `Keras` Models : ANN Example

The core objects in `Keras` are `Models` and `Layers`
- `Models` set up the container for your network
- `Layers` fill in the architecture (connections, unit types, activation functions, etc.)
- The 2 options for `Models`:
  - `Sequential`: The basic one we'll focus on
  - `Functional API`: Specify more complex models, more flexibility
- There are lots of options for types of `Layers`. We'll start with two familiar ones:
  - `Dense` layers are fully connected, meaning every node receives a linear combination of each node in its input layer.
  - `Activation` layers apply an activation function to the value of each node in the previous layer.
  - Alternatively, Dense layers may be created with an activation function as a parameter.
  
We'll look at how to build an ANN (Artificial Neural Network) using both the Sequential and Functional APIs.

## The Sequential API

Sequential Model is like a Sklearn object with extra features. Most importantly, it is an empty container that allows you to design whatever architecture you want (i.e., by adding `Layers`).

Let's start filling it with layers. We can specify all of them at the time we create the model, or we can just create an empty container and add layers to it. Each method produces the same model, but the second one can be used to build models programatically, if you're interested in going that route.

(See the [Keras Documentation on the Sequential API](https://keras.io/getting-started/sequential-model-guide/))

In [2]:
# Sequential model, all layers added at initialization
model1 = Sequential([
    Dense(32, input_dim=784),
    Activation('relu'),
    Dense(10),
    Activation('softmax'),
])

model1.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 32)                25120     
_________________________________________________________________
activation_1 (Activation)    (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                330       
_________________________________________________________________
activation_2 (Activation)    (None, 10)                0         
Total params: 25,450
Trainable params: 25,450
Non-trainable params: 0
_________________________________________________________________


In [3]:
# Sequential model, empty at initialization, layers added later.
model2 = Sequential()
model2.add(Dense(32, input_dim=784)) # 
model2.add(Activation('relu'))
model2.add(Dense(10))
model2.add(Activation('softmax'))

model2.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 32)                25120     
_________________________________________________________________
activation_3 (Activation)    (None, 32)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 10)                330       
_________________________________________________________________
activation_4 (Activation)    (None, 10)                0         
Total params: 25,450
Trainable params: 25,450
Non-trainable params: 0
_________________________________________________________________


## The Functional API

The Functional API is a different approach to building models in keras. Each layer in a network is defined as a function that takes the previous layer as an input. Then the model is created by taking the input and output layers as parameters. 

Choosing between the Sequential and Functional APIs is usually a matter of personal preference, although there are some architectures that cannot be created using the Sequential API (notably siamese networks, in which a Merge layer combines the inputs from two paths of layers).

(See the [Keras Documentation on the Functional API](https://keras.io/getting-started/functional-api-guide/))

In [4]:
# Same as the Sequential models, but needs an explicit Input layer.
# Note that each layer uses the same classes we used to create layers for our Sequential models,
# but the layers are connected by passing them as parameters in layer function calls.
# The model itself takes only the input and output layers as parameters.

input_layer = Input(shape=(784,))
hidden_layer = Dense(32)(input_layer)
hidden_act = Activation('relu')(hidden_layer)
output_layer = Dense(10)(hidden_act)
output_act = Activation('softmax')(output_layer)

model3 = Model(inputs=input_layer, outputs=output_act)
model3.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 784)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 32)                25120     
_________________________________________________________________
activation_5 (Activation)    (None, 32)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 10)                330       
_________________________________________________________________
activation_6 (Activation)    (None, 10)                0         
Total params: 25,450
Trainable params: 25,450
Non-trainable params: 0
_________________________________________________________________


In [5]:
# Activations do not need to be included as separate layers.

input_layer = Input(shape=(784,))
hidden_layer = Dense(32, activation='relu')(input_layer)
output_layer = Dense(10, activation='softmax')(hidden_layer)

model4 = Model(inputs=input_layer, outputs=output_layer)
model4.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 784)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 32)                25120     
_________________________________________________________________
dense_6 (Dense)              (None, 10)                330       
Total params: 25,450
Trainable params: 25,450
Non-trainable params: 0
_________________________________________________________________


As can be seen from the summary printouts, these three models have the same architecture. The models built with the Functional API do have the benefit of having the Input dimensions included in the summary.

# Training `Keras` Models

Sequential models and Models built with the functional API behave similarly, but there are some subtle differences. 

See the [Keras Models Documentation](https://keras.io/models/about-keras-models/) if you ever get stuck.

## Compiling the Network

The `Model.compile()` function configures a model for training. In order to do this, we need to specify the loss function, the optimizer algorithm, and the metrics we wish to use to train the model.

In [6]:
# Compile the network
model4.compile(loss='sparse_categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

## Training the Network

Similar to `sklearn`, we can train keras models by calling `fit` on `numpy` arrays.

We can specify minibatch learning using the `epochs` and `batch_size` parameters.

Below, we will load up the infamous MNIST dataset to keep our image classification theme going. 

In [11]:
from keras import datasets

# loading up the MNIST dataset
(X_train, y_train), (X_test, y_test) = datasets.mnist.load_data()

X_train = X_train.reshape(-1,784)/255 # reshape from 28x28 to 784 and
X_test = X_test.reshape(-1,784)/255   # rescale from [0,255] to [0,1]

# Fit the model 
model4.fit(X_train, y_train, epochs=10, batch_size=32)

Epoch 1/10
60000/60000 [==============================] - 3s 57us/step - loss: 0.1890 - accuracy: 0.9470
Epoch 2/10
60000/60000 [==============================] - 3s 54us/step - loss: 0.1810 - accuracy: 0.9489
Epoch 3/10
60000/60000 [==============================] - 3s 57us/step - loss: 0.1743 - accuracy: 0.9509
Epoch 4/10
60000/60000 [==============================] - 3s 56us/step - loss: 0.1681 - accuracy: 0.9529
Epoch 5/10
60000/60000 [==============================] - 3s 56us/step - loss: 0.1620 - accuracy: 0.9544
Epoch 6/10
60000/60000 [==============================] - 5s 82us/step - loss: 0.1567 - accuracy: 0.9558
Epoch 7/10
60000/60000 [==============================] - 4s 58us/step - loss: 0.1518 - accuracy: 0.9569
Epoch 8/10
60000/60000 [==============================] - 4s 59us/step - loss: 0.1472 - accuracy: 0.9586
Epoch 9/10
60000/60000 [==============================] - 4s 63us/step - loss: 0.1428 - accuracy: 0.9599
Epoch 10/10
60000/60000 [==============================

## Evaluating and Making Predictions

Trained keras models have an `evaluate` method that returns the loss values and metrics values for the model in test mode. They also have a `predict` method for making predictions.

In [8]:
# Evaluate
loss_and_metrics = model4.evaluate(X_test, y_test, batch_size=32)
print('\nLoss and Accuracy:\n', loss_and_metrics)

# Make Predictions
classes = model4.predict(X_test, batch_size=32)
#proba = model4.predict_proba(X_test, batch_size=32)
print('\nClass Predictions:\n', classes)

10000/10000 [==============================] - 0s 46us/step

Loss and Accuracy:
 [0.19392111354768277, 0.9455000162124634]

Class Predictions:
 [[2.9800573e-05 2.3430900e-07 3.7246331e-04 ... 9.9428326e-01
  5.9793372e-05 3.6907766e-04]
 [1.2797314e-03 3.1205046e-04 9.9172229e-01 ... 4.6964082e-09
  1.0021380e-03 1.5780017e-08]
 [2.0203361e-05 9.8457277e-01 3.6254532e-03 ... 6.3381498e-03
  1.2132559e-03 1.4855151e-04]
 ...
 [3.8872491e-07 4.6355896e-07 3.8032701e-06 ... 5.6317856e-04
  1.5115299e-03 1.6416235e-02]
 [6.2488056e-05 1.3805726e-04 1.5524728e-04 ... 4.8040301e-06
  2.2793479e-02 3.3265692e-06]
 [3.4258774e-05 1.0596973e-09 1.5840917e-04 ... 6.2076642e-09
  2.9072743e-07 2.8535926e-08]]


... probably not a model we'll want to put into production, but at least we're able to train it and make predictions. 

# Advanced Keras Layers

## Dropout Layers

Revisiting layers, they define:
  - Nodes (number of features in a layer, and how they are connected to nodes in the previous layer)
  - Activations (transformations applied to the data coming into a node from other nodes it is connected to) 
  - Other Properties (for example, reshaping the feature arrays, etc.)

**Dropout** is a regularization technique used to control overfitting in neural networks. Instead of adding a penalty to our loss function, Dropout works by randomly setting some proportion of the weights between layers to zero. This prevents the layer from relying too heavily on any of its inputs.

Here's an example of Dropout being used in an ANN that classifies documents from the Reuters newsgroups dataset. This approach can be used for image classification as well. 

In [9]:
# Set the max number of words to keep, we are keeping 2000 most frequent 
max_features = 2000
(X_train, y_train), (X_test, y_test) = reuters.load_data(num_words=max_features)
maxlen = 10

# Data is stored in sentences, pad any that are shorter than 10 words with zeros
X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)

model = Sequential()
# Dense(64) is a fully-connected layer with 64 hidden units.
# The 'kernel_initializer' parameter allows us to set the distribution of the
# randomized weights that the layer is initialized with.
model.add(Dense(64, input_dim=10, kernel_initializer="uniform"))
model.add(Activation('tanh'))
model.add(Dropout(0.5))
model.add(Dense(64, kernel_initializer="uniform"))
model.add(Activation('tanh'))
model.add(Dropout(0.5))
model.add(Dense(46, kernel_initializer="uniform"))
model.add(Activation('softmax'))

# Momentum: how much the weights are adjusted in response to the current gradient.
# Nesterov momentum: weights are adjusted by gradient descent, but gradient is also corrected
# after each step by examining the direction of the gradient at the new location.
sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

model.fit(X_train, y_train,
          epochs=20,
          batch_size=16)
score = model.evaluate(X_test, y_test, batch_size=16)

2113536/2110848 [==============================] - 0s 0us/step
Epoch 1/20
8982/8982 [==============================] - 2s 212us/step - loss: 4.5277 - accuracy: 0.2218
Epoch 2/20
8982/8982 [==============================] - 1s 151us/step - loss: 4.7008 - accuracy: 0.2105
Epoch 3/20
8982/8982 [==============================] - 1s 150us/step - loss: 4.8681 - accuracy: 0.2109
Epoch 4/20
8982/8982 [==============================] - 1s 138us/step - loss: 4.7827 - accuracy: 0.2133
Epoch 5/20
8982/8982 [==============================] - 1s 140us/step - loss: 4.8075 - accuracy: 0.2103
Epoch 6/20
8982/8982 [==============================] - 1s 140us/step - loss: 4.7814 - accuracy: 0.2128
Epoch 7/20
8982/8982 [==============================] - 1s 142us/step - loss: 4.6141 - accuracy: 0.2164
Epoch 8/20
8982/8982 [==============================] - 1s 157us/step - loss: 4.7575 - accuracy: 0.2152
Epoch 9/20
8982/8982 [==============================] - 1s 142us/step - loss: 4.7160 - accuracy: 0.2165
E

...based on that accuracy, not a great performance; although with 46 classes that is understandable 

## Merge Layers
- Merge multiple layer sequences into a single layer
- A number of options for merging outputs: `concat`, `sum`, `ave`, etc
- Like so:

In [10]:
left_branch_input = keras.layers.Input(shape=(784,))
x1 = keras.layers.Dense(32, )(left_branch_input)

right_branch_input = keras.layers.Input(shape=(784,))
x2 = keras.layers.Dense(32)(right_branch_input)

merged = keras.layers.concatenate([x1, x2])
main_output = keras.layers.Dense(10, activation='sigmoid')(merged)

model = keras.models.Model(inputs=[left_branch_input, right_branch_input], outputs=[main_output])

# This is what the compile and training looks like
# Note that to get this to run you will need to provide two sets of inputs, x_left and x_right
model.compile(optimizer='adam', loss='mean_squared_error')

model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 784)          0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 784)          0                                            
__________________________________________________________________________________________________
dense_10 (Dense)                (None, 32)           25120       input_2[0][0]                    
__________________________________________________________________________________________________
dense_11 (Dense)                (None, 32)           25120       input_3[0][0]                    
____________________________________________________________________________________________

## And More...

Check out the [Keras Documentation on Layers](https://keras.io/layers/about-keras-layers/) to see what your options are. Keras provides lots (and lots, and lots) of options for ways to connect nodes between layers, for transforming or normalizing data, for specifying activation functions, and so on. If you're feeling brave, [you can also write your own layers](https://keras.io/layers/writing-your-own-keras-layers/).